Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма)

In [1]:
import os
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import time

In [2]:
def yandex_mail_parse(login, password):
    
    driver = webdriver.Chrome() 
    
    driver.get("http://www.yandex.ru") 
    assert "Яндекс" in driver.title
    
    enter_button = driver.find_element_by_class_name('button')
    driver.get(enter_button.get_attribute('href'))
    
    login_input = driver.find_element_by_name('login')
    login_input.send_keys(login)
    login_input.send_keys(Keys.RETURN)
    
    password_input = WebDriverWait(driver,2).until(EC.presence_of_element_located((By.ID, 'passp-field-passwd'))) 
    password_input.send_keys(password)
    password_input.send_keys(Keys.RETURN)
    
    #mail_list = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'mail-MessageSnippet-Wrapper')))
    mail_list = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ns-view-messages-item-wrap')))
    url_list = []
    
    for mail in mail_list:
        url = mail.find_element_by_tag_name('a').get_attribute('href')
        url_list.append(url)

    pars_mail = []
    for url in url_list:
    
        mail_dict = {}
        driver.get(url)
        
        try:
            sender = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CLASS_NAME, 'mail-Message-Sender-Email'))) 
            mail_dict['sender'] = sender.text
            mail_dict['subject'] = driver.find_element_by_class_name('mail-Message-Toolbar-Subject').text
            mail_dict['date'] = driver.find_element_by_class_name('ns-view-message-head-date').text
            mail_dict['text'] = driver.find_element_by_class_name('mail-Message-Content').text

        except:
            pass
        
        driver.back()
        pars_mail.append(mail_dict) 

    driver.quit()
    
    return pars_mail

In [3]:
pars_mail = yandex_mail_parse('gt1991@yandex.ru', os.environ['PASSWD'])

In [4]:
client = MongoClient('mongodb://127.0.0.1:27017')
db = client['lesson_6']
letters = db.letters

letters.insert_many(pars_mail)

### 2) Написать программу, которая собирает «Хиты продаж» с сайтов техники mvideo, onlinetrade и складывает данные в БД. Магазины можно выбрать свои. Главный критерий выбора: динамически загружаемые товары

In [21]:
driver = webdriver.Chrome() 
driver.get("https://www.mvideo.ru/")
assert "М.Видео" in driver.title

# Избавляемся от баннера
WebDriverWait(driver,30).until(EC.presence_of_all_elements_located((By.ID, 'fl-212563')))
action = ActionChains(driver)
action.click()
action.perform()

# Получаем список кнопок для нажимания
button_list = driver.find_elements_by_class_name('carousel-paging')[2].find_elements_by_tag_name('a')

# Перемещаемся к скроллеру
action = ActionChains(driver)
action.move_to_element(driver.find_elements_by_class_name('listing-banner')[0])
action.perform()

goods_list = []  

for button in button_list:
    
    button.click()
    WebDriverWait(driver,10).until(EC.invisibility_of_element_located((By.CLASS_NAME, 'ajax-overlay_show'))) # Ожидание прогрузки товаров
    time.sleep(1) #Время на отыгрывание анимации
    
    goods_div = driver.find_elements_by_class_name('accessories-product-list')[1].find_elements_by_class_name('sel-product-tile-main')

    for iter_product in goods_div[0:4]:
        
        product = {}
        product['tile'] = iter_product.find_element_by_class_name('sel-product-tile-title').text
        product['price_current'] = iter_product.find_element_by_class_name('c-pdp-price__current').text[:-1]
        product['price_old'] = iter_product.find_element_by_class_name('c-pdp-price__old').text
        
        if product not in goods_list:
            goods_list.append(product)

driver.quit()

In [22]:
len(goods_list)

19

In [8]:
goods = db.goods

goods.insert_many(goods_list)